# 2. Clean Data
This script prepares the data for fuzzymatching. It does:
* Remove \_export_information
* Translate location (`Genève` to `Genf`)

In [8]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../../data/lib/')
import consts
import re

## Import

In [9]:
df_raw = pd.read_csv('../../data/3. transformation/1_list_all.csv')
df_data = df_raw.copy()

## Remove `_export_information`

In [10]:
df_data.drop(columns='_export_information', axis=1, inplace=True)

## Copy Zip from Location into `PLZ` and remove it in `location`
If you have an error here, you probably have addresses withouth a location. Fix that before!

In [11]:
#If there are any location empty, fix it
df_data[df_data['location'].isnull()]

,name,location,country,address,plz,uci,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,type,source,year
13973,Klinik Barmelweid,NaN,CH,Barmelweid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1200.0,1200.00,hco,daiichi,2016
13984,Quadrimed,NaN,CH,Crans-Montana,NaN,NaN,NaN,13264.00,NaN,NaN,NaN,NaN,13264.00,hco,daiichi,2016
15022,Pro Raris,NaN,CH,NaN,NaN,NaN,NaN,1500.00,NaN,NaN,NaN,NaN,1500.00,hco,hra,2016
15023,SGH Wegweiser,NaN,CH,NaN,NaN,NaN,NaN,500.00,NaN,NaN,NaN,NaN,500.00,hco,hra,2016
20828,Monika Ridinger,NaN,CH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,NaN,3000.00,hcp,vifor,2016
21453,Stiftung für Patientensicherheit in der Anästh...,NaN,CH,Rabbentalstrasse 83,NaN,NaN,0.00,5000.00,0.0,0.0,0.0,0.0,5000.00,hco,abbvie,2015
21690,"Hôpitaux Universitaires Genève, Service d' Uro...",NaN,CH,"Hôpitaux Universitaires Genève, Rue Gabrielle-...",NaN,NaN,0.00,3700.00,0.0,0.0,1000.0,0.0,4700.00,hco,astellas,2015
21693,"Hôpitaux Universitaires Genève, Service de rad...",NaN,CH,"Hôpitaux Universitaires Genève, Rue Gabrielle-...",NaN,NaN,0.00,1500.00,0.0,0.0,0.0,0.0,1500.00,hco,astellas,2015
21736,EAACI,NaN,CH,NaN,NaN,NaN,0.00,125979.45,0.0,0.0,0.0,0.0,125979.45,hco,astrazeneca,2015
21744,Foundation for Research in Rheumatology (FOREUM),NaN,CH,NaN,NaN,NaN,1067622.42,0.00,0.0,0.0,0.0,0.0,1067622.42,hco,astrazeneca,2015


In [12]:
#df_data = df_raw.copy() #REMOVE

def copy_plz(value):
    #Only check Strings
    if isinstance(value, float):
        return np.NaN
    
    r = re.search('\s?[\d]{4}\s?', value)
    if r:
        return r.group(0).strip()
    else:
        return np.NaN
#Copy into PLZ
df_data.loc[df_data.plz.isna(), 'plz'] = df_data.loc[df_data.plz.isna(), 'location'].apply(copy_plz)
df_data.loc[df_data.plz.isna(), 'plz'] = df_data.loc[df_data.plz.isna(), 'address'].apply(copy_plz)

#Clean
df_data.loc[df_data.plz.notna(), 'location'] = df_data['location'].str.replace('\s?[\d]{4}\s?', '', regex=True)
df_data.loc[df_data.plz.notna(), 'address'] = df_data['address'].str.replace(',?\s?[\d]{4}\s?', '', regex=True)

#Strip
df_data['location'] = df_data['location'].str.strip()
df_data['address'] = df_data['address'].str.strip()

## Remove Location in Address
Removed - cause of cases like:
`Kantonsspital Zürich`

In [13]:
df_data['address'] = df_data.address.astype(str)
#df_data['address'] = df_data.apply(lambda row: row['address'].replace(row['location'], ''), axis=1)

## Translate Location to «amtlich» (as the administration writes it)
`Genf` changes to `Genève`. Consistent writing.  
Source `place_translations.csv`: `Amtliches Gemeindeverzeichnis der Schweiz` https://www.bfs.admin.ch/bfs/de/home/grundlagen/agvch.html

In [14]:
#Copy dataset
#df_data = df_copy.copy() # MUSS RAUS

#Import places
df_places = pd.read_csv('sources/place_translations.csv', sep=';')

def translate_line(df_data, lng, translation):
    if isinstance(lng, str) and lng != "":
        df_data.loc[df_data.location.str.lower() == lng.lower(), 'location'] = translation
    return df_data
        
for index, row in df_places.iterrows():
    df_data = translate_line(df_data, row['de'], row['amtlich'])
    df_data = translate_line(df_data, row['fr'], row['amtlich'])
    df_data = translate_line(df_data, row['it'], row['amtlich'])


## Remove `-` in address

In [15]:
df_data.loc[df_data.address == '-', 'address'] = np.NaN
df_data.loc[df_data.address == '--', 'address'] = np.NaN

## Replace Hyphen (Viertelgeviertstrich) with minus

In [16]:
df_data = df_data.replace('‐', '-', regex=True)

## Remove `Unknown`

In [17]:
df_data = df_data.replace('Unknown / relocated', np.nan, regex=True)

## Replace NAN

In [18]:
df_data = df_data.replace('NAN', np.nan, regex=True)

## Replace `str.` with `strasse`
Not needed anymore because of libPostal Expand

In [19]:
#df_data = df_data.replace('str\.', 'strasse', regex=True)

## Replace `stras se` with `strasse`
Otherwise, `stras se` will be expanded to `stras european company`

In [20]:
df_data = df_data.replace('stras se', 'strasse', regex=True)
df_data = df_data.replace('stra sse', 'strasse', regex=True)

## Remove Comma at the end

In [21]:
df_data = df_data.replace(",\s$", '', regex=True)
df_data = df_data.replace(",$", '', regex=True)

## Save!

In [22]:
df_data.to_csv('../../data/3. transformation/2_list_all_cleaned.csv', index=False)